<a href="https://colab.research.google.com/github/Narabcs/doctorship/blob/main/fcp_nara.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [84]:
import pandas as pd
from scipy import stats
from scipy.optimize import differential_evolution as de
from scipy.integrate import quad

!git clone https://github.com/Narabcs/doctorship/

fatal: destination path 'doctorship' already exists and is not an empty directory.


### Etapa 01:

Calculando os valores de tortuosidade utilizando os dados da água
---

In [23]:
dados = pd.read_csv('/content/doctorship/parameters_water.csv')
dados['y_ress'] = dados['yexp']/dados['xexp']
dados.head()

,yexp,mi,xexp,pf,meio,y_ress
0,31500000.0,0.000891,0.0080,997.05,50,3.937500e+09
1,63000000.0,0.000891,0.0125,997.05,50,5.040000e+09
2,94500000.0,0.000891,0.0148,997.05,50,6.385135e+09
3,126000000.0,0.000891,0.0173,997.05,50,7.283237e+09
4,157000000.0,0.000891,0.0193,997.05,50,8.134715e+09


In [9]:
ress_50 = stats.linregress(x=dados.query('meio==50')['xexp'],y=dados.query('meio==50')['y_ress'])
ress_120 = stats.linregress(x=dados.query('meio==120')['xexp'],y=dados.query('meio==120')['y_ress'])

results = {'meio=50':{'r_squared':ress_50.rvalue,
           'intercept': ress_50.intercept,
           'slope': ress_50.slope},
           'meio=120':{'r_squared':ress_120.rvalue,
           'intercept': ress_120.intercept,
           'slope': ress_120.slope}
           }

results = pd.DataFrame(results)
results

,meio=50,meio=120
r_squared,9.905125e-01,9.960379e-01
intercept,1.680439e+08,2.275211e+08
slope,4.229806e+11,2.715969e+11


In [10]:
def tortuosity(slope,k,pf):
  c = slope*k**0.5/pf

  return c

In [11]:
c_50 = tortuosity(results['meio=50'].slope,1.48e-11,dados['pf'].unique())
c_120 = tortuosity(results['meio=120'].slope,3.95e-11,dados['pf'].unique())
print(f'c_50 = {float(c_50)}, c_120 = {float(c_120)}')

c_50 = 1632.0532436779447, c_120 = 1712.010470506321


### Etapa 02: 

Estimando os valores de K para as equações de Forchheimmer e Darcy.

In [22]:
dados_estimacao = pd.read_csv('/content/doctorship/parameters_onlygx.csv')
dados_estimacao.head()

,yexp,m,n,xexp,es,pf,gx,meio,c
0,31500000.0,0.774,0.326,0.00306,0.354,998.28,0.2,50,1650.0
1,63000000.0,0.784,0.321,0.00703,0.354,998.28,0.2,50,1650.0
2,94500000.0,0.762,0.325,0.01030,0.354,998.28,0.2,50,1650.0
3,126000000.0,0.767,0.322,0.01180,0.354,998.28,0.2,50,1650.0
4,157000000.0,0.771,0.317,0.01450,0.354,998.28,0.2,50,1650.0


In [14]:
def forch_opt(x, dados):
  forch = (dados[1] * ((1.1/(2.5*dados[4])**0.5) * dados[3]/x[0]**0.5)**(dados[2]-1))/x[0] * dados[3] + dados[6]*dados[5]/x[0]**0.5 * dados[3]**2
  err_rel = abs(dados[0] - forch) / dados[0] 

  return err_rel

def darcy_opt(k, dados):
  darcy = (dados[1] * ((1.1/(2.5*dados[4])**0.5) * dados[3]/k**0.5)**(dados[2]-1))/k * dados[3]
  err_rel = abs(dados[0] - darcy) / dados[0] 
  return err_rel

def modelo(dados, model:str='forch'):
  
  if model == 'forch':
    mu = (dados['m'] * ((1.1/(2.5*dados['es'])**0.5) * dados['xexp']/dados['k_forch']**0.5)**(dados['n']-1))
    y = mu/dados['k_forch'] * dados['xexp'] + dados['c']*dados['pf']/dados['k_forch']**0.5 * dados['xexp']**2

  if model == 'darcy':
    mu = (dados['m'] * ((1.1/(2.5*dados['es'])**0.5) * dados['xexp']/dados['k_darcy']**0.5)**(dados['n']-1))
    y = mu/dados['k_darcy'] * dados['xexp']

  return y, mu

In [15]:
params_bound = [(1e-20,1e-10)]

results = []

for row in dados_estimacao.iterrows():
  dados_selecionados = [row[1]['yexp'], row[1]['m'], row[1]['n'], row[1]['xexp'], row[1]['es'], row[1]['pf'], row[1]['c']]
  value = de(func=forch_opt,bounds=params_bound,args=(dados_selecionados,))
  results.append(value)

df_forch = pd.DataFrame(results)

k_bound = [(1e-20,1e-10)]

results = []

for row in dados_estimacao.iterrows():
  dados_selecionados = [row[1]['yexp'], row[1]['m'], row[1]['n'], row[1]['xexp'], row[1]['es'], row[1]['pf'], row[1]['c']]
  value = de(func=darcy_opt,bounds=k_bound,args=(dados_selecionados,))
  results.append(value)

df_darcy = pd.DataFrame(results)

In [21]:
k_forch = []
k_darcy = []

for item in zip(df_forch.x,df_darcy.x):
  k_forch.append(item[0][0])
  k_darcy.append(item[1][0])
dados_estimacao['k_forch'] = k_forch
dados_estimacao['k_darcy'] = k_darcy
dados_estimacao['dpl_forch'], dados_estimacao['mu_forch'] = modelo(dados_estimacao,model='forch')
dados_estimacao['dpl_darcy'], dados_estimacao['mu_darcy'] = modelo(dados_estimacao,model='darcy')
dados_estimacao.head()

,yexp,m,n,xexp,es,pf,gx,meio,c,k_forch,k_darcy,dpl_forch,mu_forch,dpl_darcy,mu_darcy
0,31500000.0,0.774,0.326,0.00306,0.354,998.28,0.2,50,1650.0,6.616216e-13,1.648883e-13,31500000.0,0.002711,3.150000e+07,0.001697
1,63000000.0,0.784,0.321,0.00703,0.354,998.28,0.2,50,1650.0,2.138180e-12,8.233105e-14,63000000.0,0.002229,6.300000e+07,0.000738
2,94500000.0,0.762,0.325,0.01030,0.354,998.28,0.2,50,1650.0,3.867145e-12,5.487746e-14,94500000.0,0.002117,9.450000e+07,0.000503
3,126000000.0,0.767,0.322,0.01180,0.354,998.28,0.2,50,1650.0,3.652774e-12,3.648027e-14,126000000.0,0.001857,1.260000e+08,0.000390
4,157000000.0,0.771,0.317,0.01450,0.354,998.28,0.2,50,1650.0,5.183517e-12,2.670566e-14,157000000.0,0.001748,1.570000e+08,0.000289


### Etapa 03:

Estimar a permeabilidade da torta

In [82]:
def modelo_torta(kc,dados):

  def func(x,dados):

    ps = dados[0]
    esc = dados[1]
    m = dados[2]
    n = dados[3]
    pfil = dados[4]
    const_c = dados[5]
    dP = dados[6]
    rm = dados[7]
    A = dados[8]
    dt_real = dados[11]
    alpha = dados[12]
    b_aux = dados[13]

    y = (b_aux*alpha*const_c*pfil/(A**(n+1)*dP)*x + (b_aux*rm/(A**n*dP)))**(1/n)

    return y

  # km = x[0]
  # kc = x[1]

  ps = dados[0]
  esc = dados[1]
  m = dados[2]
  n = dados[3]
  pfil = dados[4]
  const_c = dados[5]
  dP = dados[6]
  rm = dados[7]
  A = dados[8]
  V = dados[9]
  V0 = dados[10]
  dt_real = dados[11]

  # eqdarcy (dpl,mu)

  alpha = 1 / (ps*(1-esc)*kc)
  b_aux = m*(1.1/(2.5*esc)**0.5 * 1/ kc**0.5)**(n-1)

  dados.append(alpha)
  dados.append(b_aux)

  dt_cal = quad(func, V, V0, args=(dados,))

  print(dt_cal)

  erro_rel = abs(dt_real-dt_cal[0]) / dt_real

  return erro_rel

In [43]:
dados_torta = pd.read_csv('/content/doctorship/parameters_gx_caco3.csv')
dados_torta['dt_real'] = dados_torta['t']-dados_torta['t0']
dados_torta.head()

,yexp,m,n,xexp,es,esc,pfil,ps,A,Cs,t,t0,V,V0,Rm,gx,meio,dt_real
0,31500000.0,0.81,0.30,0.000004,0.354,0.510,1084.941911,2808,0.00229,0.000133,1800,602.934,0.000017,0.000004,429000000.0,0.2,50,1197.066
1,63000000.0,0.69,0.33,0.000009,0.354,0.544,1067.154343,2808,0.00229,0.000133,1800,599.136,0.000037,0.000008,429000000.0,0.2,50,1200.864
2,94500000.0,0.74,0.31,0.000010,0.354,0.516,1084.007172,2808,0.00229,0.000133,1800,599.136,0.000043,0.000005,429000000.0,0.2,50,1200.864
3,126000000.0,0.63,0.35,0.000011,0.354,0.527,1051.374826,2808,0.00229,0.000133,1800,301.056,0.000045,0.000009,429000000.0,0.2,50,1498.944
4,157000000.0,0.60,0.37,0.000011,0.354,0.524,1061.390808,2808,0.00229,0.000133,1800,159.414,0.000047,0.000008,429000000.0,0.2,50,1640.586


In [83]:
params_bound = [(1e-20,1e-10)]
# params_bound = [(1e-20,1e-10),(1,e-20,1e-10)]

results = []

for row in dados_torta.iterrows():
  dados_selecionados = [row[1]['ps'], 
                        row[1]['esc'], 
                        row[1]['m'], 
                        row[1]['n'], 
                        row[1]['pfil'], 
                        row[1]['Cs'],
                        row[1]['yexp'],
                        row[1]['Rm'],
                        row[1]['A'],
                        row[1]['V'],
                        row[1]['V0'],
                        row[1]['dt_real']]

  value = de(func=modelo_torta,bounds=params_bound,args=(dados_selecionados,))
  results.append(value)

df_torta = pd.DataFrame(results)
df_torta.head()

(-9.469909249507372e-12, 1.051371128991382e-25)
(-9.469909249507372e-12, 1.051371128991382e-25)
(-9.469909249507372e-12, 1.051371128991382e-25)
(-9.469909249507372e-12, 1.051371128991382e-25)
(-9.469909249507372e-12, 1.051371128991382e-25)
(-9.469909249507372e-12, 1.051371128991382e-25)
(-9.469909249507372e-12, 1.051371128991382e-25)
(-9.469909249507372e-12, 1.051371128991382e-25)
(-9.469909249507372e-12, 1.051371128991382e-25)
(-9.469909249507372e-12, 1.051371128991382e-25)
(-9.469909249507372e-12, 1.051371128991382e-25)
(-9.469909249507372e-12, 1.051371128991382e-25)
(-9.469909249507372e-12, 1.051371128991382e-25)
(-9.469909249507372e-12, 1.051371128991382e-25)
(-9.469909249507372e-12, 1.051371128991382e-25)
(-9.469909249507372e-12, 1.051371128991382e-25)
(-9.469909249507372e-12, 1.051371128991382e-25)
(-9.469909249507372e-12, 1.051371128991382e-25)
(-9.469909249507372e-12, 1.051371128991382e-25)
(-9.469909249507372e-12, 1.051371128991382e-25)
(-9.469909249507372e-12, 1.0513711289913

,x,fun,nfev,nit,message,success
0,[5.249189645899975e-11],1.0,32,1,Optimization terminated successfully.,True
1,[6.001746291086747e-11],1.0,32,1,Optimization terminated successfully.,True
2,[9.564449610319836e-11],1.0,32,1,Optimization terminated successfully.,True
3,[7.022889495575632e-11],1.0,32,1,Optimization terminated successfully.,True
4,[7.776602596127429e-11],1.0,32,1,Optimization terminated successfully.,True


In [28]:
k_torta = []

for item in df_torta.x:
  k_torta.append(float(item))
  
dados_torta['k_torta'] = k_torta
dados_torta.head()

,yexp,m,n,xexp,es,esc,pfil,ps,A,Cs,t,t0,V,V0,Rm,gx,meio,dt_real,k_torta
0,31500000.0,0.81,0.30,0.000004,0.354,0.510,1084.941911,2808,0.00229,0.000133,1800,602.934,0.000017,0.000004,429000000.0,0.2,50,1197.066,9.940013e-11
1,63000000.0,0.69,0.33,0.000009,0.354,0.544,1067.154343,2808,0.00229,0.000133,1800,599.136,0.000037,0.000008,429000000.0,0.2,50,1200.864,9.353011e-11
2,94500000.0,0.74,0.31,0.000010,0.354,0.516,1084.007172,2808,0.00229,0.000133,1800,599.136,0.000043,0.000005,429000000.0,0.2,50,1200.864,9.987473e-11
3,126000000.0,0.63,0.35,0.000011,0.354,0.527,1051.374826,2808,0.00229,0.000133,1800,301.056,0.000045,0.000009,429000000.0,0.2,50,1498.944,9.978466e-11
4,157000000.0,0.60,0.37,0.000011,0.354,0.524,1061.390808,2808,0.00229,0.000133,1800,159.414,0.000047,0.000008,429000000.0,0.2,50,1640.586,9.753679e-11
